In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Imports**
---



In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os
import glob
import multiprocessing as mp


pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
p = Path('/Users/alex/Dropbox/FFE')

edge_file = p / 'FinnShapeEdges.parquet'
wind_file = p / 'Copy of GD_wind.csv'
folder = p / 'output'

In [13]:
# load dataaaa
wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [ ]:
rngFile = edgelist[['source', 'IgnProbBld']]
rngFile.drop_duplicates(inplace=True)

**Definitions & parallel computing**
---

In [16]:
# num_cores = multiprocessing.cpu_count()
# print(num_cores)
# # need to do this on Shell / Terminal:
# conda install ipyparallel
# ipcluster nbextension enable --user
# ipcluster start # or ipcluster start -n 4

In [17]:
import ipyparallel as ipp
client = ipp.Client()
dview = client[:]
client.ids

[0, 1, 2, 3]

In [18]:
# add variables to all engines
dview["edgelist"]=edgelist
dview["rngFile"]=rngFile
dview["wind_data"]=wind_data
dview["folder"]=folder

# add all libraries to engines
with dview.sync_imports():
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import sys
    import os
    import glob

importing numpy on engine(s)
importing pandas on engine(s)
importing Path from pathlib on engine(s)
importing sys on engine(s)
importing os on engine(s)
importing glob on engine(s)


In [19]:
%%px

def wind_scenario(wind_data):
      import numpy as np
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(rngList=rngFile, edges=edgelist):
    rngList['rng'] = np.random.uniform(0, 1, size=rngList.values.shape[0])
    rngList = rngList[rngList['rng'] < rngList['IgnProbBld']]
    NewActiveEdges = edges[edges['source'].isin(rngList['source'])]
    return NewActiveEdges


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    import numpy as np
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    import numpy as np
    import pandas as pd
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


In [21]:
@dview.parallel(block = False) # The @parallel decorator breaks up elementwise operations and distributes them.
def ffe_runs(n):
    import numpy as np
    import pandas as pd
    for scenario in n:
        # initial setup
        listActivatedSources = []
        listScenarioDataframes = []
        condition = True
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges = ignition()
        # print(f"{len(ActiveEdges)} ignitions")
        if ActiveEdges.empty:
            continue
        while condition: # spread burn zone
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty: #no more buildings to burn
                break
            listScenarioDataframes.append(ActiveEdges)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print(f'finishing pid {os.getpid()} scenario --- {scenario} time ---- {time}')

        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["pid"] = os.getpid()
        Activations.to_parquet(str(folder) + '/' + f'scenario{scenario}_pid{os.getpid()}.parquet', engine='auto', compression="GZIP")
        

**Main**
---


---



In [22]:
%%time
ffe_runs(range(3000))

CPU times: user 3.34 ms, sys: 294 µs, total: 3.63 ms
Wall time: 3.65 ms


<AsyncMapResult: ffe_runs>


**Backup**
---



---



In [9]:
def clean_up(path=folder):
    files = path.glob('*scenario*')
    for f in files:
        print(f)
        os.remove(f)

In [10]:
clean_up()

/Users/alex/Dropbox/FFE/output/scenario2442_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario957_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1093_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2641_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario969_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1290_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1612_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2058_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1495_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2279_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2066_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1473_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2080_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2699_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1075_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1248_pid35400.parquet
/Users/alex/Dropbox/FFE/ou

/Users/alex/Dropbox/FFE/output/scenario251_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2781_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1350_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2582_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario897_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1153_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2387_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2198_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario23_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2361_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2140_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1537_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1509_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1734_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2767_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario871_pid35400.parquet
/Users/alex/Dropbox/FFE/outpu

/Users/alex/Dropbox/FFE/output/scenario1158_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2589_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1365_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1061_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1262_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1467_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2094_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1459_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1606_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1481_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2253_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2072_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1638_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario943_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1087_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2456_pid35401.parquet
/Users/alex/Dropbox/FFE/o

/Users/alex/Dropbox/FFE/output/scenario1819_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario145_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1827_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario378_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario241_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1920_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2973_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario679_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario444_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2995_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario299_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2574_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1398_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2749_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario861_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2777_pid35401.parquet
/Users/alex/Dropbox/FFE/output/

/Users/alex/Dropbox/FFE/output/scenario2204_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2007_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1412_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1695_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1014_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1229_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1217_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario316_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1849_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario115_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1877_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario328_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2824_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario710_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario513_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1891_pid35398.parquet
/Users/alex/Dropbox/FFE/outpu

...

/Users/alex/Dropbox/FFE/output/scenario1287_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1482_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1605_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2071_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2250_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1702_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2148_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2369_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1501_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2357_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2176_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2552_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario847_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1183_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2751_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario879_pid35400.parquet
/Users/alex/Dropbox/FFE/ou

/Users/alex/Dropbox/FFE/output/scenario2000_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2203_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1674_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2627_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario931_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2619_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2424_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1210_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1013_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario1692_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1415_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario514_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario729_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario717_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2823_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario112_pid35399.parquet
/Users/alex/Dropbox/FFE/outpu

/Users/alex/Dropbox/FFE/output/scenario389_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2885_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario793_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2982_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario688_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario256_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1909_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1937_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario268_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2964_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario650_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario453_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario24_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2380_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1154_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario890_pid35400.parquet
/Users/alex/Dropbox/FFE/output/sce

/Users/alex/Dropbox/FFE/output/scenario458_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario665_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1902_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario263_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario285_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2989_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario480_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario683_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1505_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2172_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario1738_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2353_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1706_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2755_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario1384_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario2568_pid35401.parquet
/Users/alex/Dropbox/FFE/output

/Users/alex/Dropbox/FFE/output/scenario2394_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario644_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario2970_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario479_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario447_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario242_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1923_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario2996_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario2891_pid35401.parquet
/Users/alex/Dropbox/FFE/output/scenario598_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario787_pid35400.parquet
/Users/alex/Dropbox/FFE/output/scenario178_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario345_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario1824_pid35398.parquet
/Users/alex/Dropbox/FFE/output/scenario146_pid35399.parquet
/Users/alex/Dropbox/FFE/output/scenario743_pid35399.parquet
/Users/alex/Dropbox/FFE/output/sce